In [65]:
import json
import pandas as pd

In [66]:
%%capture --no-stderr
%pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph nomic[local]

## Load Data

In [ ]:
# load data

try:
    with  open('data/mails.json', 'r') as f:
        email_database = json.load(f)
except FileNotFoundError:
    raise FileNotFoundError('mails.json not found')

print(f"Il y a {len(email_database)} mails dans la base de données")

try :
    actis = pd.read_csv('data/actis.csv')
except FileNotFoundError:
    raise FileNotFoundError('actis.csv not found')

print(f"Il y a {len(actis)} actis dans la base de données")

try :
    chambres = pd.read_csv('data/chambres.csv')
except FileNotFoundError:
    raise FileNotFoundError('chambres.csv not found')

print(f"Il y a {len(chambres)} chambres dans la base de données")

try :
    accueil = pd.read_csv('data/accueil.csv')
except FileNotFoundError:
    raise FileNotFoundError('accueil.csv not found')

print(f"Il y a {len(accueil)} accueils dans la base de données")

In [68]:
local_llm = "llama3.1"
local_embedder = "nomic-embed-text"

In [69]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama

embedder = OllamaEmbeddings(model = local_embedder)
llm = ChatOllama(model = local_llm, temperature=0)

## Choose where to fecth the information from

In [70]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [ ]:
PROMPT = """
Given a user question choose which datasource would be most relevant for answering their question in ["base_email", "disponibilite_chambres", "ouverture_accueil", "prix_activites", "out_of_context"].
base_email est la base de données des mails anciens
disponibilite_chambres est la base de données des dates des chambres disponibles
ouverture_accueil est la base de données des heures d'ouverture de l'accueil du logement
prix_activites est la base de données des prix des activités proposées.
out_of_context est une réponse qui n'a rien à voir avec l'hotellerie.

Même si tu sais la réponse, ne la donne pas si c'est hors contexte !!!

Tu vas répondre OBLIGATOIREMENT en json, par exemple "datasource": "base_email" ou "datasource": "out_of_context", ainsi que "question": <la question posée par l'utilisateur>.
"""


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", PROMPT),
        ("human", "{question}"),
    ]
)

from langchain_core.output_parsers import JsonOutputParser


router = {"question" : RunnablePassthrough()} | prompt | llm | JsonOutputParser()

question = "Salut, une chambre pour deux personnes est-elle disponible le 15 août ?"
router.invoke(question)

In [72]:
CHAMBRE_PROMPT = """
Tu vas utiliser le csv des chambres pour répondre à la question de l'utilisateur.
Le csv est le suivant :
""" + chambres.to_string()

dispo_chambres_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", CHAMBRE_PROMPT),
        ("human", "{question}"),
    ]
)

In [ ]:
def choose_route(result):
    if "base_email" in result['datasource']:
        return "accéder à la base d'emails"
    elif "disponibilite_chambres" in result['datasource']:
        print("accéder au CSV des chambres")
        print(result)
        return dispo_chambres_prompt
    elif "ouverture_accueil" in result['datasource']:
        return "accéder au CSV de l'accueil"
    elif "prix_activites" in result['datasource']:
        return "accéder au CSV des activités"
    elif "out_of_context" in result['datasource']:
        return "hors contexte"
    else:
        raise ValueError("datasource not recognized")

from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route) | llm

question = "Salut, une chambre pour deux personnes est-elle disponible le 15 août ?"
full_chain.invoke({"question": question})

In [ ]:
question = "combien font 2+2"
full_chain.invoke({"question": question})

In [ ]:
question = "je veux réserver une chambre frère"
full_chain.invoke({"question": question})

In [ ]:
question = "je veux faire du masque et tuba vous avez ? à quel prix ?"
full_chain.invoke({"question": question})